In [ ]:
# 导入transformers库
from transformers import BertTokenizer, BertModel

# 加载预训练的BERT模型和tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
from datasets import load_dataset

dataset= load_dataset('glue', 'sst2')
# 定义函数预处理数据集

def preprocess(example):
    # 分词
    # 使用tokenizer对输入的句子进行分词，同时进行填充和截断处理
    # 'max_length'表示填充到最大长度，'truncation=True'表示启用截断，'max_length=128'表示最大长度为128
    # 'return_tensors='pt''表示返回PyTorch张量
    tokens= tokenizer(example['sentence'], padding='max_length', truncation=True, max_length=128, return_tensors='pt')

    # 返回特征
    # 返回一个字典，包含三个键值对：
    # 'input_ids'：输入的token ID，使用squeeze()方法去除维度为1的维度
    # 'attention_mask'：注意力掩码，使用squeeze()方法去除维度为1的维度
    # 'label'：示例的标签
    return {'input_ids': tokens['input_ids'].squeeze(), 'attention_mask': tokens['attention_mask'].squeeze(), 'label': example['label']}


dataset= dataset.map(preprocess)
import torch
from torch.utils.data import DataLoader, TensorDataset

batch_size= 32
seed= 42

# 为数据集的每一个分割创建一个数据加载器

dataloaders= {}
for split in ['train', 'validation', 'test']:
    # 将特征转换为张量
    input_ids= torch.tensor(dataset[split]['input_ids'])
    attention_mask= torch.tensor(dataset[split]['attention_mask'])
    labels= torch.tensor(dataset[split]['label'])

    # 创建一个TensorDataset对象，将特征和标签组合在一起
    tensor_dataset= TensorDataset(input_ids, attention_mask, labels)

    # 创建一个数据加载器，将数据集分成多个批次
    dataloader= DataLoader(tensor_dataset, batch_size=batch_size, shuffle=split=='train', num_workers= 4)

    dataloaders[split]= dataloader

    
# 检查数据加载器是否正确创建
for split in ['train', 'validation', 'test']:
    print(f"{split} dataloader created with {len(dataloaders[split])} batches.")
# 从每个数据加载器中取一个批次
for split in ['train', 'validation', 'test']:
    print(f"Checking {split} dataloader...")
    for batch in dataloaders[split]:
        input_ids, attention_mask, labels = batch
        print(f"Input IDs shape: {input_ids.shape}")
        print(f"Attention Mask shape: {attention_mask.shape}")
        print(f"Labels shape: {labels.shape}")
        break  # 只检查一个批次
# 定义分类模型
from torch import nn

class BertSentimentClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert= BertModel.from_pretrained('bert-base-uncased')

        # 添加一个线性层
        self.classifier= nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask):
        # 使用BERT模型获取最后一层的隐藏状态
        outputs= self.bert(input_ids=input_ids, attention_mask=attention_mask)

        # 获取最后一层的隐藏状态
        cls_embeddings= outputs.last_hidden_state[:, 0, :]
        # 使用线性层进行分类
        logits= self.classifier(cls_embeddings)

        return logits
# 设置训练参数与设备

model= BertSentimentClassifier()
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer= torch.optim.AdamW(model.parameters(), lr= 2e-5)
loss_fn= nn.CrossEntropyLoss()
# 编写循环
def train_epoch(model, dataloader, loss_fn, optimizer, device):
    # 将模型设置为训练模式
    model.train()
    # 初始化总损失为0
    total_loss= 0

    # 遍历数据加载器中的每个批次
    for batch in dataloader:
        # 将输入数据、注意力掩码和标签移动到指定的设备（如GPU）
        input_ids= batch[0].to(device)
        attention_mask= batch[1].to(device)
        labels= batch[2].to(device)

        # 在每次迭代前清零优化器的梯度
        optimizer.zero_grad()

        # 通过模型前向传播得到预测的logits
        logits= model(input_ids, attention_mask)

        # 计算损失
        loss= loss_fn(logits, labels)

        # 反向传播计算梯度
        loss.backward()

        # 更新模型参数
        optimizer.step()

        # 累加当前批次的损失
        total_loss+= loss.item()

    # 返回平均损失
    return total_loss / len(dataloader)
# 验证测试函数

def evaluate(model, dataloader, loss_fn, device):
    # 将模型设置为评估模式，这样模型中的dropout和batchnorm层会工作在不同的方式
    model.eval()

    
    # 初始化总损失和正确预测的计数器
    total_loss, correct= 0, 0

    # 使用torch.no_grad()上下文管理器，这样在评估过程中不会计算梯度，节省内存和计算资源
    with torch.no_grad():
        # 遍历dataloader中的每个批次数据
        for batch in dataloader:
            # 将输入数据、注意力掩码和标签移动到指定的设备（如GPU）
            input_ids= batch[0].to(device)
            attention_mask= batch[1].to(device)
            labels= batch[2].to(device)

            # 通过模型前向传播得到预测的logits
            logits= model(input_ids, attention_mask)

            # 计算当前批次的损失
            loss= loss_fn(logits, labels)
            # 累加当前批次的损失到总损失中
            total_loss+= loss.item()

            # 获取预测结果，即logits中最大值的索引
            preds= torch.argmax(logits, dim=1)
            # 计算当前批次中预测正确的样本数，并累加到正确预测的计数器中
            correct+= (preds==labels).sum().item()
    
    acccuracy= correct / len(dataloader.dataset)
    return total_loss / len(dataloader), acccuracy
# 训练模型

epochs= 3

for epoch in range(epochs):

    train_loss= train_epoch(model, dataloaders['train'], loss_fn, optimizer, device)

    val_loss, val_accuracy= evaluate(model, dataloaders['validation'], loss_fn, device)

    print(f"Epoch: {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

In [17]:
print(model)

BertSentimentClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

In [18]:

torch.save(model.state_dict(), 'bert_sst2_model.pth')


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# 导入库（补充部分）
import torch
from torch import nn
from transformers import BertModel, AdamW

# 模型定义、训练函数、评估函数（如上述代码）

# 主流程
if __name__ == "__main__":
    # 初始化模型、优化器、设备
    model = BertSentimentClassifier().to(device)
    optimizer = AdamW(model.parameters(), lr=2e-5)
    
    # 训练循环
    for epoch in range(3):
        train_loss = train_epoch(...)
        val_loss, val_acc = evaluate(...)
        print(f"Epoch {epoch+1} Results...")
    
    # 测试评估
    test_loss, test_acc = evaluate(...)
    print(f"Final Test Accuracy: {test_acc:.4f}")